## Rešitev vseh nalog

Za vtis je tule rešitev celotnega izpita. Podrobna razlaga je spodaj.

In [10]:
from collections import defaultdict
import numpy as np


# 1 Vožnje

def kilometri(ime_dat):
    kilometrina = defaultdict(int)
    for vrstica in open(ime_dat):
        ime, km = vrstica.split(",")
        kilometrina[ime] += int(km)
    return kilometrina

def prekrskarji(ime_dat, meja):
    imena = []
    for ime, km in kilometri(ime_dat).items():
        if km > meja:
            imena.append(ime)
    return imena

def lenuh(ime_dat):
    naj_len = None
    kilometrina = kilometri(ime_dat)
    for ime, km in kilometrina.items():
        if naj_len == None or kilometrina[ime] < kilometrina[naj_len]:
            naj_len = ime
    return naj_len


# 2 Potovanje

def prevozeno(zacetek, poti):
    km = 0
    for x, y in poti:
        km += razdalja(zacetek, x) + razdalja(x, y)
        zacetek = y
    return km


# 3 Deja vu

def stevilo_vozenj(voznik, vozniki, razdalje):
    return np.sum(vozniki == voznik)

def prevozena_razdalja(voznik, vozniki, razdalje):
    return np.sum(razdalje[vozniki == voznik])

def povprecje_treh(voznik, vozniki, razdalje):
    return np.mean(np.sort(razdalje[vozniki == voznik])[-3:])


# 4 

def voznje_voznika(voznik, razdalje, casi):
    for razdalja, cas in zip(razdalje, casi):
        print(f"{voznik:10}{razdalja:10} km{cas:10} min{razdalja / (cas / 60):10.1f} km/h")
        voznik = ""
    print("-" * 52)

def voznje_vseh(voznje):
    razdalje = defaultdict(list)
    casi = defaultdict(list)
    for voznik, razdalja, cas in voznje:
        razdalje[voznik].append(razdalja)
        casi[voznik].append(cas)

    print("-" * 52)
    for voznik in sorted(razdalje):
        voznje_voznika(voznik, razdalje[voznik], casi[voznik])


# 5

def e_kamion(pot, polnilnice, kapaciteta):
    baterija = kapaciteta
    for odkod, kam in pairwise(pot):
        baterija -= razdalja(odkod, kam)
        if baterija < 0:
            return odkod
        if kam in polnilnice:
            baterija = kapaciteta
    return kam

## 1. Vožnje

Vožnje so shranjene v datotekah v takšni obliki:

```
Ana,15
Berta,42
Ana,85
Berta,83
Cilka,15
Berta,29
Berta,82
Dani,81
Cilka,192
Dani,92
Ana,115
Ema,200
```

Vsaka vrstica vsebuje ime voznika in razdaljo. Napišite naslednje funkcije:

- `kilometri(ime_datoteke)` vrne slovar, katerega ključi so imena voznikov, pripadajoče vrednosti pa skupna dolžina voženj, ki jih je opravil ta voznik. Za primer iz okvirčka vrne `{'Ana': 215, 'Berta': 236, 'Cilka': 207, 'Dani': 173, 'Ema': 200}`.
- `prekrskarji(ime_datoteke, meja)` vrne seznam imen voznikov, ki so prevozili več kot `meja` kilometrov. Klic `prekrskarji("voznice.txt", 200)` vrne `["Ana", "Berta", "Cilka"]`.
- `lenuh(ime_datoteke)` vrne ime voznika, ki je skupno prevozil najmanj kilometrov, v gornjem primeru `"Dani"`. Če je več enako lenih, vrne kateregakoli od njih.

### Rešitev

To je tako podobno vsem dražbam, ki smo jih predelovali na predavanjih, da ne bi smelo biti pretežko.

In [1]:
def kilometri(ime_dat):
    kilometrina = defaultdict(int)
    for vrstica in open(ime_dat):
        ime, km = vrstica.split(",")
        kilometrina[ime] += int(km)
    return kilometrina

def prekrskarji(ime_dat, meja):
    imena = []
    for ime, km in kilometri(ime_dat).items():
        if km > meja:
            imena.append(ime)
    return imena

def lenuh(ime_dat):
    naj_len = None
    kilometrina = kilometri(ime_dat)
    for ime, km in kilometrina.items():
        if naj_len == None or kilometrina[ime] < kilometrina[naj_len]:
            naj_len = ime
    return naj_len

S to nalogo študenti večinoma res niso imeli problemov. Všeč mi je bilo tudi, da so mnogi, ki sicer niso pravilno napisali prve funkcije, vseeno uspešno napisali drugi dve. Tretja je lahko seveda tudi drugačna (lahko, recimo, shranjujemo najkrajšo dolžino v ločeno spremenljivko), pomembno pa je, da ne izgubljamo časa s ponovnim pisanjem branja datoteke, temveč lepo pokličemo prvo funkcijo.

## 2. Potovanje

V testih je že napisana funkcija `razdalja(odkod, kam)`, ki vrne razdaljo med dvema krajema. Klic `razdalja("Kranj", "Ljubljana")` vrne 26. Klic `razdalja("Kranj", "Kranj")` vrne `0`. **Te funkcije ne pišite, ta že obstaja!**

Napišite funkcijo `prevozeno(zacetek, poti)`, ki prejme ime nekega kraja, v katerem se nahaja voznik, in seznam relacij (v obliki parov krajev), ki jih mora ta voznik prevoziti. Klic `prevozeno("Postojna", [("Ljubljana", "Kranj"), ("Kranj", "Novo Mesto"), ("Lendava", "Ormož")])` vrne `461`. Do te številke pridemo tako: voznik mora najprej iz Postojne v Ljubljano. Nato prevozi (podano) relacijo Ljubljana – Kranj, nato prevozi (podano) relacijo Kranj – Novo Mesto (saj se le-ta začne v Kranju in voznik je že tam), nato gre iz Novega Mesta v Lendavo(!!!), da pride v Lendavo, ker mora za konec dneva peljati še iz Lendave v Ormož. (Funkcija je nekajkrat krajša od tega opisa.)

### Rešitev

Iti moramo čez zaporedne pare krajev na poti, kar, vemo, naredimo s `pairwise(pot)`. Za vsak par pokličemo `razdalja` in to lepo seštevamo. Manjša težava je v tem, da moramo vsakič priti iz začetnega - ali prejšnjega končnega kraja - v novi kraj, torej je potrebno dodati še to razdaljo.

In [3]:
def prevozeno(zacetek, poti):
    km = 0
    for x, y in poti:
        km += razdalja(zacetek, x) + razdalja(x, y)
        zacetek = y
    return km

Nekateri so namesto ene zanke naredili dve: v prvi so iz seznama parov sestavili nov seznam, ki v katerem so bili kraji (v bistvu) podvojeni, nato pa so računali pare med temi kraji. No, če tale stavek zveni zapleteno: ni, vendar je takšna rešitev vseeno bolj zapletena, kot bi bilo potrebno. :)

## 3. Déjà vu

Naslednje funkcije prejmejo tri argumente: ime voznika in tabeli v numpyju, ki vsebujeta stolpca, kakršna imajo datoteke iz prve naloge, torej `["Ana", "Berta", "Ana", "Berta", "Cilka", ...]` in `[15, 42, 85, 83, 15, 29 ...]`. Uporabite, kar potrebujete. Naloge rešujte z numpyjem; drugačne rešitve bodo prejele polovično število točk.

- `stevilo_vozenj(voznik, vozniki, razdalje)` vrne število voženj, ki jih je opravil podani voznik.
- `prevozena_razdalja(voznik, vozniki, razdalje)` vrne skupno razdaljo, ki jo je prevozil podani voznik
- `povprecje_treh(voznik, vozniki, razdalje)` vrne povprečno razdaljo najdaljših treh voženj. Če je voznik opravil le dve vožnji, vrne njuno povprečno dolžino; če le eno, vrne dolžino te.

### Rešitev

Vse funkcije so dobile tri argumente; iz čiste hudobije. Prva potrebuje le dva, saj se ne ukvarja z razdaljami. Na to sem sicer namignil tudi v besedilu naloge.

Izvzemši to hudobijo je naloga sestavljena na moč prijazno, saj je narejena v obliki stopničk: vsaka naslednja funkcija temelji na ideji, ki jo moramo imeti za prejšnjo. Tako do zadnje funkcijo pridemo lepo po stopničkah.

Rešitve so takšne:

In [5]:
def stevilo_vozenj(voznik, vozniki, razdalje):
    return np.sum(vozniki == voznik)

def prevozena_razdalja(voznik, vozniki, razdalje):
    return np.sum(razdalje[vozniki == voznik])

def povprecje_treh(voznik, vozniki, razdalje):
    return np.mean(np.sort(razdalje[vozniki == voznik])[-3:])

Prva funkcija je uvodna in nas napelje na to, da se splača sestaviti `vozniki == voznik`; ta ideja nam bo prišla prav tudi v prihodnjih dveh nalogah. (Nekateri so tu in v naslednjih funkcijah klicali tudi `flatnonzero`; to je nepotrebno.) V prvi, torej, dobimo, tabelo `True`-jev in `False`-ov, pri čemer `True`-ji ustrezajo elementom, ki so enaki iskanem vozniku. Ker je `True` toliko kot `1` in `False` toliko kot `0`, je vsota te tabele ravno število voženj tega voznika.

V drugi funkciji to tabelo uporabimo kot masko v tabelo razdalj in jo seštejemo.

V tretji te razdalje uredimo in izračunamo povprečje zadnjih treh. Če jih je manj kot 3, bo tabela pač krajša. `np.mean` se ne bo jezil niti, če dobi tabelo dolžine 1.

## 4. Poročila

Napišite naslednji funkciji.

- `voznje_voznika(voznik, razdalje, casi)` prejme ime voznika, seznam razdalj voženj tega voznika in čas v minutah, ki ga je potreboval za posamično vožnjo, npr. `voznje_voznika("Peter", [102, 124, 24, 110], [55, 67, 16, 89])`. Funkcija takšno tabelo.

    ```
    Peter            102 km        55 min     111.3 km/h
                     124 km        67 min     111.0 km/h
                      24 km        16 min      90.0 km/h
                     110 km        89 min      74.2 km/h
    ----------------------------------------------------
    ```

    Ime in številke so izpisane na 10 mest, minusov je ravno prav. Hitrost izračunate tako, da razdaljo delite s časom v urah.

- `voznje_vseh(voznje)`, ki prejme seznam trojk (voznik, razdalja, cas), na primer `[('Peter', 102, 55), ('Tine', 24, 23), ('Tone', 215, 206), ('Peter', 124, 68), ('Peter', 24, 13), ('Tine', 24, 16) ... in tako naprej]`. Funkcija mora izpisati poročilo v takšni obliki:

    ```
    ----------------------------------------------------
    Andrej            34 km        18 min     113.3 km/h
                      52 km        42 min      74.3 km/h
    ----------------------------------------------------
    Ervin            245 km       159 min      92.5 km/h
    ----------------------------------------------------
    Peter            102 km        55 min     111.3 km/h
                     124 km        68 min     109.4 km/h
                      24 km        13 min     110.8 km/h
                     110 km        89 min      74.2 km/h
    ----------------------------------------------------
    ```

### Rešitev

Prva funkcija zahteva le (preprosto!) oblikovanje nizov. No, pa na `zip` se je dobro spomniti. Brez tega pa se moramo znajti s kakim indeksiranjem.

In [6]:
def voznje_voznika(voznik, razdalje, casi):
    for razdalja, cas in zip(razdalje, casi):
        print(f"{voznik:10}{razdalja:10} km{cas:10} min{razdalja / (cas / 60):10.1f} km/h")
        voznik = ""
    print("-" * 52)

Da bi bila naloga vsaj malo začinjena, ime voznika izpišemo le v prvi vrsti. Večina študentov je za to štela vrstice, tule pa smo naredili preprosteje: po prvem izpisu smo zamenjali ime voznika s praznim nizom.

Druga naloga je bila v resnici naloga iz slovarjev (ker prej ni bilo praktično še nobene, vsaj ne s slovarji v glavni vlogi), ne iz izpisa. Bistvo naloge je bilo v tem, da odkrijemo, da so podatki za voznike pomešani, mi pa jih želimo izpisovati po voznikih, zato je potrebno nabrati skupaj podatke za vsakega voznika.

Pripravimo si lahko dva slovarja: slovar z dolžinami voženj in slovar s časi za vsakega voznika. Ključi, torej, so imena voznikov, pripadajoče vrednosti pa seznami dolžin in seznami časov. Zakaj dva seznama namesto seznama parov? Zato, da lahko izkoristimo prejšnjo funkcijo, ki zahteva takšna seznama.

In [7]:
def voznje_vseh(voznje):
    razdalje = defaultdict(list)
    casi = defaultdict(list)
    for voznik, razdalja, cas in voznje:
        razdalje[voznik].append(razdalja)
        casi[voznik].append(cas)

    print("-" * 52)
    for voznik in sorted(razdalje):
        voznje_voznika(voznik, razdalje[voznik], casi[voznik])

## 5. Trajnostno-zeleni brezogljični kamion

Napišite funkcijo `e_kamion(pot, polnilnice, kapaciteta)`, ki prejme `pot` (seznam krajev), po kateri naj bi šel kamion, seznam krajev, v katerih je kamion možno polniti (argument polnilnice), in kapaciteto baterije, torej število kilometrov, ki jih lahko prevozi med dvema polnjenjema baterije.

Kamion ima v začetku polno baterijo. Napolni jo v vsakem kraju iz seznama pot, ki ima polnilnico. Če trenutni nivo baterije ne zadošča za pot do naslednjega kraja, se kamion ustavi. (Kako dobiti razdalje med kraji, preberite v drugi nalogi.) Funkcija naj vrne kraj, v katerem se kamion ustavi, oziroma zadnji kraj, če lahko prevozi celotno pot.

### Rešitev

Vsak izpit ima praviloma še eno nalogo iz zank in pogojev. Tokrat je bila to zadnja naloga. Ni bila pretirano zapletena, le razmisliti je bilo potrebno in to misel zapisati v Pythonu.

V začetku je tank poln (`tank = kapaciteta`). Nato za vsak par krajev v `pot` (`for odkod, kam in pairwise(pot)`) zmanjšamo napolnjenost baterije za razdaljo med krajema (`baterija -= razdalja(odkod, kam)`). Če je v bateriji potem manj kot nič elektrike, očitno ne pridemo v `kam`, torej je pot končana; vrnemo `odkod`. Sicer pa smo uspešno prispeli v `kam` in če je tam polnilnica, napolnimo baterijo.

Če se zanka konča, vrnemo (zadnji) `kam`).

In [8]:
def e_kamion(pot, polnilnice, kapaciteta):
    baterija = kapaciteta
    for odkod, kam in pairwise(pot):
        baterija -= razdalja(odkod, kam)
        if baterija < 0:
            return odkod
        if kam in polnilnice:
            baterija = kapaciteta
    return kam